In [14]:
# !pip install pyspark

In [15]:
!pip install mmh3


In [1]:
# !apt-get update
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [21]:
# import pyspark
# from pyspark.sql import DataFrame, SparkSession
# from pyspark.ml.feature import Tokenizer, RegexTokenizer
# from pyspark.sql.functions import col
# import hashlib
# from pyspark.sql.functions import lower, regexp_replace, split, udf
# from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType

Initialiser Apache Spark

In [22]:
# spark = SparkSession.builder \
#     .master("local[*]") \
#     .appName("Document Similarity") \
#     .getOrCreate()
# sc = spark.sparkContext


In [2]:
import mmh3

**Classe Shingling(Spark)**

In [24]:
# # class Shingling():
# #     def __init__(self, k=10):
# #         """
# #         k: length of shingles (in words)
# #         """
# #         self.k = k

# #     def clean_text(self, df: DataFrame, text_col: str) -> DataFrame:
# #         # Mettre en minuscule, supprimer caractères spéciaux, réduire espaces multiples
# #         df = df.withColumn(text_col, lower(col(text_col)))
# #         df = df.withColumn(text_col, regexp_replace(col(text_col), r"[^a-z0-9\s]", ""))
# #         df = df.withColumn(text_col, regexp_replace(col(text_col), r"\s+", " "))
# #         return df

# #     def tokenize(self, df: DataFrame, text_col: str) -> DataFrame:
# #         df = df.withColumn("tokens", split(col(text_col), "\\s+"))

# #         def clean_tokens(tokens):
# #             # Supprimer None et chaînes vides
# #             return [str(t).strip() for t in tokens if t and str(t).strip()]

# #         clean_udf = udf(clean_tokens, ArrayType(StringType())) #udf(...) transforme clean_tokens en fonction distribuée Spark.#Spark ne peut pas directement appliquer des fcts Python, besoin d'un wrapper -> udf
# #         df = df.withColumn("tokens", clean_udf(col("tokens")))
# #         return df

# #     def generate_shingles(self, df: DataFrame) -> DataFrame:
# #         k = self.k

# #         def shingle_function(tokens):
# #             tokens = [str(t).strip() for t in tokens if t and str(t).strip()]
# #             if len(tokens) < k:
# #                 return []
# #             shingles = [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]
# #             # Supprimer shingles vides ou None
# #             return [s for s in shingles if s and s.strip()]

# #         shingle_udf = udf(shingle_function, ArrayType(StringType()))
# #         df = df.withColumn("shingles", shingle_udf(col("tokens")))
# #         return df

# #     def hash_shingles(self, df: DataFrame) -> DataFrame:
# #         def hash_list(shingles):
# #             hashed = []
# #             for s in shingles:
# #                 if s and s.strip():
# #                     h = mmh3.hash64(s.strip(), signed=False)[0] #retourne entier non signé entre 0 et 2^64 -1
# #                     h = h % (2**63)  # forcer des entier entre 0 et 2^63 -1 #Sinon j'ai eu un overflow pour LongType et donc des valeurs NULL de hashage (mon prob initial)
# #                     #h = mmh3.hash("some string", signed=False)  # 32-bit unsigned integer
# #                     hashed.append(h)
# #             return hashed

# #         hash_udf = udf(hash_list, ArrayType(LongType()))   #longtype entre - 2^63 et 2^63 -1
# #         df = df.withColumn("hashed_shingles", hash_udf(col("shingles")))
# #         return df

# #     def transform(self, df: DataFrame, text_col: str) -> DataFrame:
# #         df = self.clean_text(df, text_col)
# #         df = self.tokenize(df, text_col)
# #         df = self.generate_shingles(df)
# #         df = self.hash_shingles(df)
# #         return df

# class Shingling():
#     def __init__(self, k=10):
#         self.k = k

#     def transform(self, df: DataFrame, text_col: str) -> DataFrame:
#         from pyspark.sql.functions import col, lower, regexp_replace, split, udf
#         from pyspark.sql.types import ArrayType, LongType, StringType
#         import mmh3

#         # Nettoyage du texte
#         df = df.withColumn(text_col, lower(col(text_col)))
#         df = df.withColumn(text_col, regexp_replace(col(text_col), r"[^a-z0-9\s]", ""))
#         df = df.withColumn(text_col, regexp_replace(col(text_col), r"\s+", " "))

#         # Tokenization
#         df = df.withColumn("tokens", split(col(text_col), "\\s+"))

#         # Génération des shingles
#         def shingles_fn(tokens):
#             k = self.k
#             tokens = [str(t).strip() for t in tokens if t and str(t).strip()]
#             if len(tokens) < k:
#                 return []
#             return [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]

#         shingle_udf = udf(shingles_fn, ArrayType(StringType()))
#         df = df.withColumn("shingles", shingle_udf(col("tokens")))

#         # Hash des shingles
#         def hash_fn(shingles):
#             hashed = []
#             for s in shingles:
#                 if s and s.strip():
#                     h = mmh3.hash64(s.strip(), signed=False)[0]
#                     h = h % (2**63)
#                     hashed.append(h)
#             return hashed

#         hash_udf = udf(hash_fn, ArrayType(LongType()))
#         df = df.withColumn("hashed_shingles", hash_udf(col("shingles")))

#         # Garder seulement filename et hashed_shingles
#         df = df.select("filename", "hashed_shingles")
#         return df


In [3]:
import re
import mmh3

class Shingling:
    def __init__(self, k=10):
        self.k = k  # nombre de mots par shingle

    def clean_text(self, text: str) -> str:
        text = text.lower()
        text = re.sub(r"[^a-z0-9\s]", " ", text)
        text = re.sub(r"\s+", " ", text).strip()
        return text

    def tokenize(self, text: str):
        return text.split()

    def generate_shingles(self, tokens):
        k = self.k
        return [" ".join(tokens[i:i+k]) for i in range(len(tokens) - k + 1)]

    def hash_shingles(self, shingles):
        hashed = [mmh3.hash64(s, signed=False)[0] % (2**63) for s in shingles]
        return hashed

    def process_document(self, text: str):
        cleaned = self.clean_text(text)
        tokens = self.tokenize(cleaned)
        shingles = self.generate_shingles(tokens)
        hashed = self.hash_shingles(shingles)
        return hashed


**Classe CompareSets**

In [4]:
class CompareSets:
    @staticmethod
    def jaccard_similarity(set1, set2):
        if not set1 or not set2:
            return 0.0
        s1, s2 = set(set1), set(set2)
        return len(s1 & s2) / len(s1 | s2)


In [5]:
import random
class MinHashing():
  def __init__(self,num_hashes = 100,max_hash = 2**32 -1):
    self.num_hashes = num_hashes
    self.max_hash = max_hash
    self.hash_params = [(random.randint(1,self.max_hash -1),random.randint(0,self.max_hash-1)) for _ in range(num_hashes)]

  def ComputeSignatures(self,shingle_set):
    signature = []
    for (a,b) in self.hash_params:
      min_hash = min(((a*x + b)% self.max_hash) for x in shingle_set)
      signature.append(min_hash)
    return signature




In [6]:
shingles_doc1 = [6840193950337781150, 1695454882927409067, 7334696573069238721]
minhasher = MinHashing()
signature = minhasher.ComputeSignatures(shingles_doc1)

print(signature)



[825999843, 99439174, 1727863141, 141968942, 889074749, 3039606286, 2214678375, 1755671392, 52988319, 670821659, 582138174, 1493498556, 105392070, 2398914357, 711477627, 400207598, 2397516307, 595553760, 966339065, 2000690835, 1022071868, 1076582195, 2555345081, 1711784201, 307679545, 1314178635, 710824964, 3057683080, 48244578, 537393149, 1381948069, 228143124, 1307964398, 142082879, 818600845, 378778544, 626786850, 804932739, 61963376, 160258214, 1669587846, 2465295608, 306579501, 1260370449, 1769445955, 725765453, 590983094, 2374342158, 734578666, 111111468, 614692668, 889399249, 1592978693, 1415859935, 912682788, 1518048219, 1510305959, 103159389, 1960351194, 2671309449, 1017753898, 1112661943, 1771693509, 2068917019, 2328089292, 2123313183, 84293296, 1154031320, 2332659265, 79120369, 2033298130, 1656704445, 49766138, 1515914670, 451066912, 2121860713, 1185588738, 2256632871, 1948910719, 1710909440, 1307099233, 446893784, 1840000704, 893773593, 141276491, 996803956, 1650557963, 245

In [7]:
class CompareSignatures():
  @staticmethod
  def compare(sig1,sig2):
    if len(sig1) != len(sig2):
      raise ValueError("Signatures must have the same length")

    matches = sum(1 for a,b in zip(sig1,sig2) if a == b)
    return matches /len(sig1)


**Test à part**

In [37]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col
# import mmh3
# from itertools import combinations

# # --- Spark session ---
# spark = SparkSession.builder.master("local[*]").appName("Test MinHash").getOrCreate()

# # --- Exemples de textes ---
# docs = [
#     ("doc1.txt", "The quick brown fox jumps over the lazy dog."),
#     ("doc2.txt", "The quick brown fox jumps over the lazy dog!"),  # très similaire à doc1
#     ("doc3.txt", "Alice was beginning to get very tired of sitting by her sister on the bank."),
#     ("doc4.txt", "It was the best of times, it was the worst of times."),
#     ("doc5.txt", "Mr. Utterson, the lawyer, was a man of a rugged countenance.")
# ]

# # --- Création du DataFrame Spark ---
# df_docs = spark.createDataFrame(docs, ["filename", "text"])

# # --- Shingling (comme ta classe Shingling) ---
# shingler = Shingling(k=3)  # 3 mots par shingle pour exemple
# df_shingles = shingler.transform(df_docs, "text")

# # --- MinHashing ---
# n_hashes = 50
# minhasher = MinHashing(n_hashes)
# df_shingles = df_shingles.withColumn("minhash", minhasher.minhash_udf()(col("hashed_shingles")))

# # --- Comparaison MinHash ---
# threshold_minhash = 0.6
# rows = df_shingles.select("filename", "minhash").collect()
# similar_pairs_minhash = []

# for r1, r2 in combinations(rows, 2):
#     sim = CompareSignatures.compare(r1["minhash"], r2["minhash"])
#     if sim >= threshold_minhash:
#         similar_pairs_minhash.append((r1["filename"], r2["filename"], sim))

# print("Pairs with MinHash similarity >= 0.6:")
# for f1, f2, sim in similar_pairs_minhash:
#     print(f"{f1} ~ {f2} : {sim:.2f}")


Pairs with MinHash similarity >= 0.6:
doc1.txt ~ doc2.txt : 1.00


In [ ]:
from collections import defaultdict
from itertools import combinations
class LSH():
  def __init__(self,num_bands=20,num_rows_band=5):
    self.num_bands = num_bands
    self.num_rows_band = num_rows_band
    self.buckets = [defaultdict(list) for _ in range(num_bands)]

  def band_hash(self,band):
    return mmh3.hash(str(band),signed = False)
  
  def index(self, doc_signatures, doc_names):
    for doc_id,sig in zip(doc_names,doc_signatures):
      for b in range(self.num_bands):
        start = b*self.num_rows_band
        end = b + self.num_rows_band
        band = tuple(sig[start:end])
        h = self.band_hash(band)
        self.buckets[b][h].append(doc_id)

  def candidate_pairs(self):
    """
    Retourne la liste des paires candidates (documents qui tombent dans au moins un même bucket)
    """
    candidates = set()
    for b in range(self.num_bands):
      for bucket_docs in self.buckets[b].values():
        if len(bucket_docs) > 1:
                    # Toutes les combinaisons de documents dans le bucket
          for pair in combinations(bucket_docs, 2):
            candidates.add(tuple(sorted(pair)))
    return list(candidates)
